<a href="https://colab.research.google.com/github/Pritam-Mondal18/ai-Comic-Crafter/blob/main/comicCrafterFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.5 MB/s eta 0:00:00


In [2]:
pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 6.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.8-cp311-cp311-linux_x86_64.whl size=5959638 sha256=ff56dfd2c0dc577ef0c665b07faf492623888e78e472da996f18bdbd4341ebf5
  Stored in directory: /root/.cache/pip/wheels/c0/03/66/eb3810eafd55d921b2be32896d1f44313996982360663aa80b
Successfully built llama-cpp-python


In [3]:
from huggingface_hub import login
login("YOUR_ACCESS_TOKEN")


In [4]:
%%writefile app.py
import streamlit as st
import torch
from diffusers import StableDiffusionXLPipeline
from transformers import pipeline
import os
import zipfile
from io import BytesIO

# Set Page Configuration
st.set_page_config(page_title="AI Comic Crafter", page_icon="🦸", layout="wide")

# Load Stable Diffusion Model (Hugging Face Diffusers)
@st.cache_resource
def load_sd_model():
    model_id = "stabilityai/stable-diffusion-xl-base-1.0"
    pipe = StableDiffusionXLPipeline.from_pretrained(
        model_id,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
    )
    pipe.to("cuda" if torch.cuda.is_available() else "cpu")
    return pipe

pipe = load_sd_model()

# Load a Smaller LLM Model (GPT-2 for Text Generation)
@st.cache_resource
def load_llm():
    return pipeline("text-generation", model="gpt2")

llm = load_llm()

# Custom Comic-Style CSS with Dark Theme
comic_css = """
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Comic+Neue:wght@700&display=swap');
        html, body, [class*="st-"] {
            font-family: 'Comic Neue', cursive;
            background-color: #1a1a2e;
            color: #ffffff;
        }
        .title {
            text-align: center;
            color: #ffcc00;
            font-size: 52px;
            text-shadow: 4px 4px 0px #000;
        }
        .comic-box {
            border: 4px solid #ff4500;
            padding: 20px;
            background: #222;
            box-shadow: 5px 5px 0px #ff4500;
            margin: 15px;
            border-radius: 10px;
        }
    </style>
"""
st.markdown(comic_css, unsafe_allow_html=True)

# Title
st.markdown("<h1 class='title'>🎨 AI Comic Crafter 🦸</h1>", unsafe_allow_html=True)

# Sidebar Settings
st.sidebar.header("📜 Story Settings")
genre = st.sidebar.selectbox("Choose a Genre", ["Superhero", "Fantasy", "Sci-Fi", "Mystery"])
style = st.sidebar.selectbox("Comic Style", ["Manga", "Cartoon", "Realistic", "Vintage"])
num_panels = 4  # Fixed to match the 4 story sections

# Story Input
st.markdown("<div class='comic-box'><h2>📝 Enter Your Story Elements:</h2></div>", unsafe_allow_html=True)
introduction = st.text_area("Introduction", "")
storyline = st.text_area("Storyline", "")
climax = st.text_area("Climax", "")
moral = st.text_area("Moral", "")

# Generate Story Using LLM
if st.button("🤖 Generate Story with AI"):
    with st.spinner("Generating story elements..."):
        intro_text = llm(f"Generate an introduction for a {genre} comic story.", max_length=50)[0]['generated_text']
        story_text = llm(f"Generate the main storyline for a {genre} comic story.", max_length=100)[0]['generated_text']
        climax_text = llm(f"Generate a climax for a {genre} comic story.", max_length=50)[0]['generated_text']
        moral_text = llm(f"Generate a moral for a {genre} comic story.", max_length=40)[0]['generated_text']

    introduction = st.text_area("Introduction", intro_text)
    storyline = st.text_area("Storyline", story_text)
    climax = st.text_area("Climax", climax_text)
    moral = st.text_area("Moral", moral_text)

# Function to Generate Comic-Style Images
def generate_comic_images(prompts):
    image_paths = []
    for i, prompt in enumerate(prompts):
        panel_prompt = f"{prompt}, {style} comic-style, highly detailed, vibrant colors"
        image = pipe(prompt=panel_prompt).images[0]  # Generate image
        image_path = f"comic_panel_{i+1}.png"
        image.save(image_path)
        image_paths.append(image_path)
    return image_paths

# Generate Story & Images
if st.button("🦸 Generate Comic"):
    st.markdown("<h2 style='color:#ffcc00'>📖 Story Panels:</h2>", unsafe_allow_html=True)

    with st.spinner("Generating Comic Panels... 🖌️"):
        image_paths = generate_comic_images([introduction, storyline, climax, moral])

    story_parts = ["Introduction", "Storyline", "Climax", "Moral"]
    for i, img_path in enumerate(image_paths):
        st.markdown(f"<div class='comic-box'><h3><b>{story_parts[i]}:</b> {eval(story_parts[i].lower())}</h3>", unsafe_allow_html=True)
        st.image(img_path, caption=f"Comic Panel {i+1}")
        st.markdown("</div>", unsafe_allow_html=True)

    # Zip and provide download button
    if image_paths:
        zip_buffer = BytesIO()
        with zipfile.ZipFile(zip_buffer, "w") as zip_file:
            for img_path in image_paths:
                zip_file.write(img_path)
        zip_buffer.seek(0)
        st.sidebar.download_button(
            label="📥 Download All Panels",
            data=zip_buffer,
            file_name="comic_panels.zip",
            mime="application/zip"
        )

# Footer
st.sidebar.write("✨ Powered by AI & Pritam 🚀")


Writing app.py


In [5]:
!pkill -f ngrok


In [6]:
!ngrok authtoken YOUR_ACCESS_TOKEN

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
from pyngrok import ngrok
import os

# Run Streamlit in the background
os.system("streamlit run app.py &")

# Expose port 8501 (Streamlit's default port)
public_url = ngrok.connect(8501)
print(f"Streamlit app is live at: {public_url}")

Streamlit app is live at: NgrokTunnel: "https://d325-34-19-84-118.ngrok-free.app" -> "http://localhost:8501"
